# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import seaborn as sns
import warnings

from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import (
    StratifiedKFold, 
    KFold,
    cross_validate,
)

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    make_scorer,
    precision_recall_curve,
)

from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.paths import MODELS_DIR

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

import mlflow
import mlflow.xgboost

In [3]:
# Execução do modelo

mlflow.set_experiment("Fraud Detection XGBoost Experiment")
with mlflow.start_run():

   # Importando a base
    transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

    # Embaralha as transações para garantir aleatoriedade
    transacoes = transacoes.sample(frac=1, random_state=42)  

    # Criando os escaladores
    power_transformer = PowerTransformer(method='yeo-johnson')  
    minmax_scaler = MinMaxScaler()

    # Aplicando a normalização nas colunas específicas
    transacoes['Amount'] = power_transformer.fit_transform(transacoes[['Amount']])
    transacoes['Time'] = minmax_scaler.fit_transform(transacoes[['Time']])

    # Separando X e y
    X = transacoes.drop(columns='Class')
    y = transacoes['Class']

    # KFold com 10 dobras
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Variáveis para armazenar as métricas de cada dobra
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    # Adição do MLflow: Log de Parâmetros 
    # Log dos hiperparâmetros do modelo. 
    params = {
        "max_depth": 9,
        "n_estimators": 200,
        "learning_rate": 0.3,
        "random_state": 0,
        "scale_pos_weight": (y.value_counts()[0] / y.value_counts()[1]),
    }
    mlflow.log_params(params)

    # Para iterar pelas dobras
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Treinando o modelo XGBoost
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)
        
        # Fazendo previsões
        y_pred = model.predict(X_test)
        
        # Calculando as métricas
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_test, y_pred, zero_division=0))

    # Calculando a média das métricas
    mean_accuracy = np.mean(accuracy_scores)
    mean_precision = np.mean(precision_scores)
    mean_recall = np.mean(recall_scores)

    # Exibindo os resultados médios
    print("Média das métricas após 10 dobras:")
    print(f"Acurácia: {mean_accuracy:.4f}")
    print(f"Precisão: {mean_precision:.4f}")
    print(f"Recall: {mean_recall:.4f}")

    # Adição do MLflow: Log de Métricas e Registro do Modelo 
    # Log das métricas finais
    mlflow.log_metric("mean_accuracy", mean_accuracy)
    mlflow.log_metric("mean_precision", mean_precision)
    mlflow.log_metric("mean_recall", mean_recall)

    # Captura um exemplo de entrada para a assinatura do modelo.
    # Usamos o primeiro registro de X para garantir que a estrutura seja a mesma.
    input_example = X_test.iloc[[0]]

    # Registra o modelo com o nome correto e o exemplo de entrada.
    mlflow.xgboost.log_model(
        xgb_model=model, 
        name="best_model", 
        input_example=input_example
    )

Média das métricas após 10 dobras:
Acurácia: 0.9996
Precisão: 0.9227
Recall: 0.8102
